# Project

# Data

The dataset<sup><a name="note-cariboo" href="#cariboo">[7]</a></sup> is composed of 7 different subsets, we will be using 4 of the datasets. Since each feature resides in different sets. 

- `Appearances.csv`
    - Player ID, Game ID, Appearance ID, Competition ID, Player club ID, Assist, Minutes Played, Yellow cards, Red Cards
- `Clubs.csv `
    - Club ID, Name, Pretty_name, Domestic_competition_id, Total_market_value, Squad_size, Average_age, Foreigners_numbers, Foreigners_percentage, National_team_players, Stadium_name, Stadium_seats, Net_transfer_record, Coach_name, URL
- `Competitions.csv`
    - Competition_id, Name, type, country_id, country_name, domestic_league_code, confederation, URL.  
- `Games.csv` 
    - Game_id, Competition_code, Season, Round, Date, Home_club_id, Away_club_id, Home_club_goals, away_club_goals, Home_club_postions, Away_club_postion, Stadium, Attendance, Referee, URL 
- `Leagues.csv` 
    - League_id, name, Confederation
- `Player_valuations.csv` 
    - Player_id, Date, Market_value
- `Players.csv`
    - Player_id, Last_season, Current_club_id, Name, Pretty_name, country_of_birth, Country_of_citizenship, Date_of_birth, Position, Sub_position, Foot, Height_in_cm, Market_value_in_gbp, Highest_market_value_in_gbp, URL

- _What an observation consists of_: We are trying to use the variables we assume to be the most important and independent from each other. We decided on 
    - Club, Nationality, League, Appearances, Minutes, Goals, Assist, Age, Conduct, Media Presence, Years Played, Trophies, Experience, Position, Physicality, Potential. 
- _What some critical variables are, how they are represented_: We want variables which have the highest co-variance with each other. The metric should handle most features as unique features. 
- _Any special handling, transformations, cleaning, etc will be needed_: There will be club names, and probably inferences in our data. Such as Media Presence or Potential, these are metrics which can be objective to the person. How popular is the player that we are analyzing? 

We are still going to be in search of more databases that might have different descriptive data that we might like to see how organizations search for talent. We can use what they might describe as their most sought out characteristics. 

For Simplicity we can also assume that all players have no contracts for their evaluation and are based solely on performance and the other variables mentioned.

In [1]:
!{__import__('sys').executable} -m pip install --quiet pandas

In [2]:
import pandas as pd

_data_ = {
    name: pd.read_csv(
        file, 
        engine = 'c',
        low_memory = True,
        memory_map = False, # set `False` to load into memory
        **kwargs
    ) for name, file, kwargs in [
        ('appearances', 'data/appearances.csv', {}),
        ('clubs', 'data/clubs.csv', {}),
        ('competitions', 'data/competitions.csv', {}),
        ('games', 'data/games.csv', {}),
        ('leagues', 'data/leagues.csv', {}),
        ('players', 'data/players.csv', {
            'parse_dates': ['date_of_birth'],
            'dtype': {
                'last_season': 'category',
                'country_of_birth': 'category',
                'country_of_citizenship': 'category',
                'position': 'category',
                'sub_position': 'category'
            }
        }),
        ('player_valuations', 'data/player_valuations.csv', {})
    ]
}

In [3]:
data = {}

In [4]:
data['clubs'] = _data_['clubs'].copy()

data['clubs'] = data['clubs'][['club_id', 'name']]
data['clubs'].rename(
    columns = {'name': 'club_name'},
    inplace = True
)
data['clubs'].set_index('club_id', inplace = True)

data['clubs']

,club_name
club_id,
1032,fc-reading
2323,orduspor
1387,acn-siena-1904
3592,kryvbas-kryvyi-rig
1071,wigan-athletic
...,...
1269,pec-zwolle
200,fc-utrecht
317,fc-twente-enschede


In [5]:
data['games'] = _data_['games'].copy()

data['games'] = data['games'][[
    'season', 
    'game_id', 
    'home_club_id', 
    'away_club_id'
]]
data['games'].set_index('game_id', inplace = True)

data['games']

,season,home_club_id,away_club_id
game_id,,,
2244388,2012,131,418
2219794,2011,58,498
2244389,2012,418,131
2271112,2012,48,60
2229332,2012,27,16
...,...,...,...
3646190,2021,84351,1053
3646188,2021,5791,20108
3655616,2021,5874,12275


In [6]:
data['appearances'] = _data_['appearances'].copy()

data['appearances'] = data['appearances'].loc[
    :, ~data['appearances'].columns.isin([
        'appearance_id', 
        'competition_id'
    ])
]
data['appearances'].rename(
    columns = {'player_club_id': 'club_id'},
    inplace = True
)

data['appearances'] = data['appearances'].merge(
    data['games'], 
    on = 'game_id',
    copy = False
)
data['appearances'].drop(columns = 'game_id', inplace = True)

data['appearances'] = (
    data['appearances']
        .groupby(['player_id', 'season'])
        .agg({
            **{
                c: 'sum' for c in [
                    'goals', 
                    'assists', 
                    'minutes_played', 
                    'yellow_cards', 
                    'red_cards'
                ]
            },
            'club_id': 'last',
            'home_club_id': 'last',
            'away_club_id': 'last'
        })
)

data['appearances'].reset_index('season', inplace = True)

data['appearances']

,season,goals,assists,minutes_played,yellow_cards,red_cards,club_id,home_club_id,away_club_id
player_id,,,,,,,,,
10,2014,16,9,2289,6,0,398,398,9690
10,2015,8,7,1714,3,0,398,4102,398
26,2014,0,0,2880,0,0,16,15,16
26,2015,0,0,1260,1,0,16,16,1058
26,2016,0,0,1020,0,0,16,16,167
...,...,...,...,...,...,...,...,...,...
926954,2021,0,0,24,1,0,2778,2778,5724
926987,2021,0,0,3,0,0,1053,1053,206
934034,2021,0,0,3,0,0,60949,3385,60949


In [7]:
data['player_valuations'] = _data_['player_valuations'].copy()

data['player_valuations']['season'] = \
    pd.DatetimeIndex(data['player_valuations']['date']).year
data['player_valuations'].drop(columns = 'date', inplace = True)

data['player_valuations'] = (
    data['player_valuations']
        .groupby(['player_id', 'season'])
        .agg({'market_value': 'mean'})
)
data['player_valuations'].reset_index('season', inplace = True)

data['player_valuations'].rename(
    columns = {'market_value_in_gbp': 'market_value'},
    inplace = True
)

data['player_valuations']

,season,market_value
player_id,,
10,2004,6300000.0
10,2005,10800000.0
10,2006,22500000.0
10,2007,20700000.0
10,2008,18000000.0
...,...,...
928831,2021,45000.0
928831,2022,45000.0
934034,2021,45000.0


In [8]:
data['players'] = _data_['players'].copy()

data['players'] = data['players'].loc[
    :, ~data['players'].columns.isin([
        'last_season',
        'pretty_name',
        'current_club_id',
        'market_value_in_gbp',
        'highest_market_value_in_gbp',
        'country_of_birth',
        'url', 
        'foot'
    ])
]
data['players'].rename(
    columns = {
        'height_in_cm': 'height',
        'country_of_citizenship': 'nationality'
    },
    inplace = True
)

data['players'].set_index('player_id', inplace = True)

data['players']

,name,nationality,date_of_birth,position,sub_position,height
player_id,,,,,,
254016,arthur-delalande,France,1992-05-18,Midfield,midfield - Central Midfield,186
51053,daniel-davari,Iran,1988-01-06,Goalkeeper,Goalkeeper,192
31451,torsten-oehrl,Germany,1986-01-07,Attack,attack - Centre-Forward,192
44622,vladimir-kisenkov,Russia,1981-10-08,Defender,Defender - Right-Back,182
30802,oscar-diaz,Spain,1984-04-24,Attack,attack - Centre-Forward,183
...,...,...,...,...,...,...
462285,fabian-de-keijzer,Netherlands,2000-05-10,Goalkeeper,Goalkeeper,193
368612,merveille-bokadi,DR Congo,1996-05-21,Defender,Defender - Centre-Back,186
408574,joey-veerman,Netherlands,1998-11-19,Midfield,midfield - Central Midfield,185


In [9]:
data['all'] = data['players'].merge(
    data['player_valuations'].merge(
        data['appearances'], 
        on = ['player_id', 'season'],
        copy = False
    ), 
    on = 'player_id',
    copy = False
)
'''
.merge(
    data['clubs'], 
    on = 'club_id',
    copy = False
)
data['all'].drop(columns = 'club_id', inplace = True)
'''

data['all']['age'] = (
    pd.to_datetime(data['all']['season'], format = '%Y', utc = True) 
        - pd.to_datetime(data['all']['date_of_birth'], utc = True)
).astype('timedelta64[Y]')
data['all'].drop(columns = 'date_of_birth', inplace = True)

data['all']

,name,nationality,position,sub_position,height,season,market_value,goals,assists,minutes_played,yellow_cards,red_cards,club_id,home_club_id,away_club_id,age
player_id,,,,,,,,,,,,,,,,
9800,artem-milevskyi,Ukraine,Attack,attack - Centre-Forward,189,2020,90000.0,0,2,618,3,0,6131,6131,282,34.0
43084,gaetano-berardi,Switzerland,Defender,Defender - Right-Back,179,2020,360000.0,0,0,114,0,0,399,399,984,31.0
230826,gennaro-acampora,Italy,Midfield,midfield - Central Midfield,174,2020,360000.0,1,2,624,2,0,3522,3522,130,25.0
198087,matteo-ricci,Italy,Midfield,midfield - Defensive Midfield,176,2020,1530000.0,0,3,2440,5,0,3522,4083,3522,25.0
110689,deniz-mehmet,Turkey,Goalkeeper,Goalkeeper,192,2020,68000.0,0,0,540,0,0,1519,3016,1519,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364245,jordan-teze,Netherlands,Defender,Defender - Centre-Back,183,2019,420000.0,0,0,180,0,0,383,383,2761,19.0
364245,jordan-teze,Netherlands,Defender,Defender - Centre-Back,183,2020,1102500.0,0,1,3747,5,0,383,141,383,20.0
364245,jordan-teze,Netherlands,Defender,Defender - Centre-Back,183,2021,5400000.0,1,4,2630,6,0,383,383,132,21.0


In [10]:
data['all'] = data['all'].merge(
        data['clubs'], 
        on = ['club_id'],
        copy = False)


In [11]:
data['all'] = data['all'].drop(columns = ['club_id','home_club_id','away_club_id'])

In [12]:
data['all']

,name,nationality,position,sub_position,height,season,market_value,goals,assists,minutes_played,yellow_cards,red_cards,age,club_name
0,gaetano-berardi,Switzerland,Defender,Defender - Right-Back,179,2020,360000.0,0,0,114,0,0,31.0,leeds-united
1,pablo-hernandez,Spain,Attack,midfield - Attacking Midfield,173,2020,1170000.0,0,2,417,1,0,34.0,leeds-united
2,barry-douglas,Scotland,Defender,Defender - Left-Back,176,2020,1950000.0,0,0,90,0,0,30.0,leeds-united
3,eunan-okane,Ireland,Midfield,midfield - Central Midfield,168,2016,1350000.0,0,0,118,0,0,25.0,leeds-united
4,matt-grimes,England,Midfield,midfield - Central Midfield,178,2016,1800000.0,0,0,217,1,0,20.0,leeds-united
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50267,guillaume-dietsch,France,Goalkeeper,Goalkeeper,184,2020,292500.0,0,0,180,0,0,18.0,rfc-seraing
50268,guillaume-dietsch,France,Goalkeeper,Goalkeeper,184,2021,675000.0,0,0,2782,2,1,19.0,rfc-seraing
50269,timothy-galje,Belgium,Goalkeeper,Goalkeeper,180,2021,90000.0,0,0,90,1,0,19.0,rfc-seraing
50270,antoine-bernier,Belgium,Attack,attack - Left Winger,170,2020,240000.0,0,0,180,0,0,22.0,rfc-seraing


In [13]:
data['all'][data['all']['name'] == 'cristiano-ronaldo']

,name,nationality,position,sub_position,height,season,market_value,goals,assists,minutes_played,yellow_cards,red_cards,age,club_name
8003,cristiano-ronaldo,Portugal,Attack,attack - Centre-Forward,187,2018,96000000.0,28,10,3646,4,1,32.0,juventus-turin
8004,cristiano-ronaldo,Portugal,Attack,attack - Centre-Forward,187,2019,74250000.0,35,7,3991,3,0,33.0,juventus-turin
8005,cristiano-ronaldo,Portugal,Attack,attack - Centre-Forward,187,2020,54000000.0,38,4,3841,5,0,34.0,juventus-turin
24833,cristiano-ronaldo,Portugal,Attack,attack - Centre-Forward,187,2021,39000000.0,24,3,3101,10,0,35.0,manchester-united
45293,cristiano-ronaldo,Portugal,Attack,attack - Centre-Forward,187,2014,96000000.0,61,23,4641,6,1,28.0,real-madrid
45294,cristiano-ronaldo,Portugal,Attack,attack - Centre-Forward,187,2015,105000000.0,51,15,4293,3,0,29.0,real-madrid
45295,cristiano-ronaldo,Portugal,Attack,attack - Centre-Forward,187,2016,99000000.0,42,12,4126,5,0,30.0,real-madrid
45296,cristiano-ronaldo,Portugal,Attack,attack - Centre-Forward,187,2017,90000000.0,44,8,3678,5,0,31.0,real-madrid


In [14]:
lis1t =data['all']['sub_position'].unique()
pd.DataFrame(lis1t)

,0
0,Defender - Right-Back
1,midfield - Attacking Midfield
2,Defender - Left-Back
3,midfield - Central Midfield
4,Goalkeeper
5,attack - Right Winger
6,Defender - Centre-Back
7,attack - Centre-Forward
8,attack - Second Striker
9,attack - Left Winger


In [ ]:
data['all']['name']= data['all']['name'].apply(lambda x: x.title(), )
data['all']['name']= data['all']['name'].apply(lambda x: x.replace('-', ' '))
data['all']['club_name']= data['all']['club_name'].apply(lambda x: x.title(), )
data['all']['club_name']= data['all']['club_name'].apply(lambda x: x.replace('-', ' '))
# data['all']['sub_position']= data['all']['sub_position'].apply(lambda x: x.title(), )
# data['all']['sub_position']= data['all']['sub_position'].apply(lambda x: x.replace('Defender', ''))
# data['all']['sub_position']= data['all']['sub_position'].apply(lambda x: x.replace('attack', ''))
# data['all']['sub_position']= data['all']['sub_position'].apply(lambda x: x.replace('midfield', ''))
# data['all']['sub_position']= data['all']['sub_position'].apply(lambda x: x.replace('Midfield', ''))
# data['all']['sub_position']= data['all']['sub_position'].apply(lambda x: x.replace('-', ' '))


data['all']

In [ ]:
#one hot encode categorical features
data['all'] = pd.get_dummies(data['all'], columns = ['position', 'sub_position', 'nationality','club_name'])
data['all']

In [ ]:
data['all'].dtypes

In [ ]:
data['all'].describe()

In [ ]:
data['all'][data['all']['name'] == 'lionel-messi']

# Evaluation Metrics

We will be using an OLS regression model and the evaluation techniques we are considering are RMSE and Euclidean distance.
A possible evaluation metric we will use is RMSE or Mean Absolute Value of Errors. It is derived by calculating the difference between the estimated and actual value, square those results, then calculate the mean of those results. The formula for RMSE is 

$$
    \text{RMSE} = \sqrt{\frac{\sum_{i = 1}^{N}\left(\text{Predicted}_{i} - \text{Actual}_{i}\right)^{2}}{N}}
$$

# Preliminary results

In [ ]:
!{__import__('sys').executable} -m pip install --quiet statsmodels numpy seaborn

In [ ]:
# OLS using statsmodels
import statsmodels.api as sm
import numpy as np

In [ ]:
# feature selection
X = data['all'][[
    'goals', 
    'assists', 
    'minutes_played', 
    'yellow_cards', 
    'red_cards', 
    'height'
   # 'nationality',
   # 'position',
   # 'sub_position',
   # 'club_name'
]]
y = data['all']['market_value']

X = sm.add_constant(X)

OLS_model = sm.OLS(y, X).fit()
OLS_model.summary()

# X coeffecients and constant terms for prediction/plot
params = OLS_model.params

print(OLS_model.summary())
print(params)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

import seaborn as sns

In [ ]:
sns.displot(data = data['all'], x = 'market_value', bins = 50)

plt.show()

In [ ]:
df_highest_market_value_players = data['all'].nlargest(n = 1, columns = 'market_value')

df_highest_market_value_players

In [ ]:
df_highest_market_value = data['all'].loc[data['all']['name'].isin(df_highest_market_value_players['name'])]

df_highest_market_value

In [ ]:
sns.barplot(data = df_highest_market_value, x = 'season', y = 'market_value')
plt.title('Player Market Value Over the Years')
plt.xticks(rotation = 45, ha = 'right', rotation_mode = 'anchor')

plt.show()

# Footnotes

<sup><a name="cariboo" href="#note-cariboo">7</a></sup> *Football Data from Transfermarkt.* (2022, April 22). [Dataset]. https://www.kaggle.com/datasets/davidcariboo/player-scores